# **Microsoft Defender Threat Intelligence**

## Jupyter NoteBook using the MDTI to return the Reputation DataSet.  There is an example of a single domain/ip address entity and then an example of a bulk Reputation return for classification as well as score.

#### Author: 
Dennis Mercer  
Sr Program Manager  
Microsoft CxE MDTI  

##### Microsoft Defender Threat Intelligence (MDTI) is a complete threat intelligence platform that enables security professionals to ingest, analyze and act upon trillions of signals collected from across the internet and processed by security experts and AI. MDTI allows users to uncover and understand the global threat landscape as it relates to their organization, including context around vulnerabilities, threat actors, and the threat infrastructure that might be used to attack them.

##### Microsoft processes over 65-trillion signals every day, which is used to populate the database which MDTI pulls datasets of threat intelligence. The key to operationalizing threat intelligence is to make it not just applicable, but actionable.   This notebook seeks to make threat intelligence actionable for SOC Analyst, Threat Hunters and Cyber Threat Intelligence Researchers.

##### With this Jupyter Notebook, you can call the Reputation Endpoint and return the reputation classification and score.   Additionally, you can use additional APIs to create additional datasets for your notebook.



_________________________________________________________________________________________________________________________________________________________
### You will need to install aiohttp and msal for the bulk Reputation request.

In [ ]:
pip install aiohttp msal

________________________________________________________________________________________________________________________________________________________
### Import Statements for any potential calls within the cells below or any modifications that could be made to the source of the Typosquat Services

In [ ]:
import asyncio
from azure.identity.aio import ClientSecretCredential
from kiota_authentication_azure.azure_identity_authentication_provider import AzureIdentityAuthenticationProvider
from msgraph import GraphRequestAdapter
from msgraph import GraphServiceClient
from IPython.display import display
import ipywidgets as widgets
from azure.identity import ClientSecretCredential
import requests
import pandas as pd
import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import json

### This cell will return the JSON response for a single entity (Domain or IP Address).

In [ ]:
# Get the client secret from a local file
credential = ClientSecretCredential(
    tenant_id='Enter your TenantID',     # Tenant ID
    client_id='Enter your App ClientID', # ClientID from app registration 
    client_secret=  client_secret        # You should not leave the client secret exposed, delete the secret once you have used it or employ Key Vault
)

scopes = ['https://graph.microsoft.com/.default']  # Scopes or permissions required for API access

auth_provider = AzureIdentityAuthenticationProvider(credential, scopes=scopes)
request_adapter = GraphRequestAdapter(auth_provider)
client = GraphServiceClient(request_adapter)

# Updated get_reputation function
async def get_reputation(hostname: str):
    reputation = await client.security.threat_intelligence.hosts.by_host_id(hostname).reputation.get()
    if reputation:
        classification = str(reputation.classification).split('.')[-1]  # Get only the classification part
        print(f"The domain you entered is {classification} with a score of {reputation.score}")
    else:
        print(f"No reputation information found for {hostname}.")

try:
    loop = asyncio.get_running_loop()
except RuntimeError:  # 'RuntimeError: There is no current event loop...'
    loop = None

entity_to_check = input("Enter the domain for which you want to retrieve the reputation score: ")  # Replace with the entity you want to check

if loop and loop.is_running():
    loop.create_task(get_reputation(entity_to_check))
else:
    asyncio.run(get_reputation(entity_to_check))


### This cell will return a bulk Reputation query and return the Reputation Classification and Score for each entity.  The query uses an Asynchronous method to call the API Endpoint.  You will need to list the entities that you wish to query in a text file, each entity on a separate line and then add the path to that file in the code.

In [47]:
# Use the client secret to create the credential
credential = ClientSecretCredential(
    tenant_id='Enter your TenantID',     # Tenant ID
    client_id='Enter your App ClientID', # ClientID from app registration 
    client_secret=  client_secret        # You should not leave the client secret exposed, delete the secret once you have used it or employ Key Vault
)

scopes = ['https://graph.microsoft.com/.default']  # Scopes or permissions required for API access

auth_provider = AzureIdentityAuthenticationProvider(credential, scopes=scopes)
request_adapter = GraphRequestAdapter(auth_provider)

client = GraphServiceClient(request_adapter)

# Updated get_reputation function
async def get_reputation(hostname: str):
    try:
        reputation = await client.security.threat_intelligence.hosts.by_host_id(hostname).reputation.get()
        if reputation:
            classification = str(reputation.classification).split('.')[-1]  # Get only the classification part
            print(f"The domain {hostname} is {classification} with a score of {reputation.score}")
    except Exception as e:
        print(f"Error occurred while checking reputation for domain {hostname}: {e}")

try:
    loop = asyncio.get_running_loop()
except RuntimeError:  # 'RuntimeError: There is no current event loop...'
    loop = None

# Read list of entities from the text file with domains separated by a carriage return
file_path = r'C:\Users\dennismercer\Documents\Python\MDTI\token.txt'

try:
    with open(file_path, 'r', encoding='utf-16') as file:
        entities_to_check = [line.strip() for line in file.readlines()]
except UnicodeDecodeError:
    with open(file_path, 'r', encoding='utf-8-sig') as file:
        entities_to_check = [line.strip() for line in file.readlines()]

for entity in entities_to_check:
    if entity:  # Skip empty lines
        print(f"Checking domain: {entity}")
        if loop and loop.is_running():
            tsk = loop.create_task(get_reputation(entity))
        else:
            result = asyncio.run(get_reputation(entity))


Checking domain: 062fe.com
Checking domain: hostelhotels.net
Checking domain: microsoft.com
Checking domain: riskiq.net


The domain 062fe.com is Suspicious with a score of 74
The domain microsoft.com is Unknown with a score of None
The domain riskiq.net is Unknown with a score of None
The domain hostelhotels.net is Malicious with a score of 100
